In [12]:
from operator import add
from scipy import spatial
import glob
import sys
import numpy
from scipy.spatial import distance
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import random 
import heapq
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
model= Doc2Vec.load("d2v.model")

In [14]:
def get_sentences(tag):
    files = glob.glob("Extraction-and-Summarization-of-Tweets/data/tweets/"+tag+"/*")
    sentences = []
    for file in files:
        fd = open(file,"r")
        for line in fd:
            sentences.append(line)
    return sentences
        

In [15]:
def get_vectors(sentences):
    sen_vectors = []
    for sentence in sentences:
        test_data = word_tokenize(sentence.lower())
        v1 = model.infer_vector(test_data)
        sen_vectors.append(v1)
    return sen_vectors
        
    

In [16]:
def ranking(sentence_vectors,sentences):
    centroid_vector = [0 for i in range(len(sentence_vectors[0]))]
    l = len(sentences)
    for vector in sentence_vectors:
        centroid_vector = list(map(add, centroid_vector, vector))
    rank = []
    count = 0
    for vector in sentence_vectors:
        rank.append([cosine_similarity([vector,centroid_vector])[0][1],sentences[count]])
        count+=1
    return rank , list(numpy.array(centroid_vector)/l) 


In [17]:
def beam_search(candidate_set,vectors,theta,k,dm_avg):
    l = len(candidate_set)
    lk = []
    lold = []
    lnew = []
    for j in range(k): 
        lold.append([candidate_set[random.randint(0, l)][1]])
    #print(lold)
    while len(lold):
        for sentence in candidate_set:
            for summary_set in lold:
                if sentence[1] not in summary_set:
                    summary_set = summary_set + [sentence[1]]
                    test_data = word_tokenize('.'.join(summary_set).lower())
                    v1 = model.infer_vector(test_data)
                    error = (distance.euclidean(v1, dm_avg))**2 
                    if len(summary_set) < theta:
                        lnew.append([error,summary_set])
                        lnew = heapq.nsmallest(k,lnew)
                    elif len(summary_set) == theta:
                        lk.append([error,summary_set])
                        lk = heapq.nsmallest(k,lk)
        
        lold = [row[1] for row in lnew]
        lnew = []
    
    return '.'.join(lk[0][1])
                        
                        
                
        
    
    
    

In [18]:
hashtag = sys.argv[1]

In [19]:
sentences = get_sentences("metoo")


In [20]:
sen_vectors = get_vectors(sentences)

In [21]:
candidate_set,dm_avg = ranking(sen_vectors,sentences)

In [22]:


summary = beam_search(candidate_set,sen_vectors,5,5,dm_avg) 


In [23]:
summary

'"Be careful," other women whispered to me when they found out I was working on projects with Rev.@BudHeckman. Now, 4 amazingly strong and brave women are bringing his behavior to and they must hold him accountable. .It\'s false accusations like this that hurt Ali could just as well be victim here as was Johnny Depp etc. Learn to hear both sides of the story for the movement to succeed. .If HR is such a vital component of American business, asks how did it miss the kind of sexual harassment at the center of the movement? .Suddenly, our body\'s in fight, flight, or freeze. With our heart racing, we face frightful memories. I\'ve learned we cannot outrun trauma. Despite how many years I\'ve tried to escape the mental torment, I realized the only way to heal is to feel the pain. .The cost of speaking up. How many deaths have taken place in Pakistan and elsewhere? When women stand up to sexual harassment and are slapped with defamation suits, penalties and intimidation. '